In [1]:
url_tmpl = "https://issue.pbsbiotech.com/projects/%s/issues.csv?utf8=%%E2%%9C%%93&columns=all"
_p_urls = [
    "pbscustomer", "pbsdisposables", "pbsinstruments", 
    "magic-metals", "manufacturing", "pbssoftware", "swtesting",
    "system-qualification-testing"
]
project_urls = [url_tmpl % p for p in _p_urls]
project_urls

['https://issue.pbsbiotech.com/projects/pbscustomer/issues.csv?utf8=%E2%9C%93&columns=all',
 'https://issue.pbsbiotech.com/projects/pbsdisposables/issues.csv?utf8=%E2%9C%93&columns=all',
 'https://issue.pbsbiotech.com/projects/pbsinstruments/issues.csv?utf8=%E2%9C%93&columns=all',
 'https://issue.pbsbiotech.com/projects/magic-metals/issues.csv?utf8=%E2%9C%93&columns=all',
 'https://issue.pbsbiotech.com/projects/manufacturing/issues.csv?utf8=%E2%9C%93&columns=all',
 'https://issue.pbsbiotech.com/projects/pbssoftware/issues.csv?utf8=%E2%9C%93&columns=all',
 'https://issue.pbsbiotech.com/projects/swtesting/issues.csv?utf8=%E2%9C%93&columns=all',
 'https://issue.pbsbiotech.com/projects/system-qualification-testing/issues.csv?utf8=%E2%9C%93&columns=all']

In [1]:
import pyquery
import requests
import urllib

In [3]:
s = requests.Session()


def cookie2string(cookie):
    return ";".join("=".join((k,v)) for k, v in cookie.items())

def string2cookie(string):
    s = string.split(",")
    cookie = {}
    for c in s:
        c = c.strip()
        try:
            k, v = c.split("=", 1)
        except:
            pass
        else:
            cookie[k] = v
    return cookie

def update_cookies(oldh, rsph):
    if 'set-cookie' not in rsph:
        return
    c = string2cookie(oldh.get('Cookie', ""))
    new = string2cookie(rsph.get('set-cookie', ""))
    c.update(new)
    s = cookie2string(c)
    oldh['Cookie'] = s
    
def dh(h):
    for k, v in h.items():
        print("%s=%s" % (k,v))

headers = {}
headers['Connection'] = 'keep-alive'
headers['User-Agent'] = "Python Httplib2"
headers['Accept'] = 'text/html, application/xhtml+xml,*/*'
h, c = s.request("https://issue.pbsbiotech.com", "GET", None, headers)
update_cookies(headers, h)
assert h.status == 200
#os.startfile("myhtml.html")
q = pyquery.PyQuery(c)

kvs = {}
for e in q("#login-form :input"):
    d = e.attrib
    if 'name' in d:
        k = d['name']
        v = d.get('value', "")
        kvs[k]=v
        
kvs['username'] = 'nstarkweather@pbsbiotech.com'
kvs['password'] = 'kookychemist'
del kvs['autologin']
body = "&".join("=".join((k,v)) for k, v in kvs.items())
body = urllib.parse.urlencode(kvs)

h,c=s.request("https://issue.pbsbiotech.com/login", "POST", body.encode("ascii"), headers)
print(h.status)
update_cookies(headers, h)

MissingSchema: Invalid URL 'GET': No schema supplied. Perhaps you meant http://GET?

In [10]:
import requests
import urllib
import pyquery
from collections import OrderedDict

uj = urllib.parse.urljoin

class IssuetrackerAPI():
    _login_url = "login"
    _proj_url = "projects/%s/issues"
    _issues_url = "issues"
    def __init__(self, base_url):
        r = urllib.parse.urlparse(base_url)
        if not r.scheme and not r.netloc:
            base_url = urllib.parse.urlunparse(("https", r.path, "", r.params, r.query, r.fragment))
        #base_url = base_url.rstrip("/")
        self._base_url = base_url
        self._sess = requests.Session()
        self._headers = {}
        self._username = None
    
    def login(self, user='Admin', pw='hunter2'):
        r1 = self._sess.get(self._base_url)
        r1.raise_for_status()
        q = pyquery.PyQuery(r1.content)
        data = {}
        for td in q("#login-form :input"):
            at = td.attrib
            if 'name' in at:
                k = at['name']
                v = at.get('value', "")
                data[k] = v
        data['username'] = user
        data['password'] = pw
        body = urllib.parse.urlencode(data)
        r2 = self._sess.post(uj(self._base_url, self._login_url), body)
        r2.raise_for_status()
        if not pyquery.PyQuery(r2.content)("#loggedas"):
            raise ValueError("Invalid Username or Password")
        return r2
        
    def download_project_csv(self, project, utf8=True, columns='all'):
        if utf8:
            utf8 = "%E2%9C%93"
        else:
            utf8 = ""
        url_end = ".csv?utf8=%s&columns=%s" 
        url = (self._proj_url + url_end ) % (project, utf8, columns)
        url = uj(self._base_url, url) + ""
        r = self._sess.get(url)
        r.raise_for_status()
        return self._parse_issues(r.content)
    
    def download_issue(self, id, type='pdf'):
        assert type in ('pdf', '.pdf'), "Non-pdf download not supported"
        name = "%d%s" % (id, ".pdf")
        url = uj(self._base_url, self._issues_url + "/" + name)
        r = self._sess.get(url)
        r.raise_for_status()
        return r.content
        
    def _parse_issues(self, csv, encoding='utf-8'):
        if not isinstance(csv, str):
            csv = csv.decode(encoding)
        sl = csv.splitlines()
        sl[0] = sl[0].lower().replace('"', "")
        lines = [l.split(",") for l in sl]
        issues = OrderedDict()
        for l in lines[1:]:
            issue = _Issue(self)
            for key, val in zip(lines[0], l):
                issue[key] = val.strip('"') or "<n/a>"
            issue['#'] = int(issue['#'])
            issues[issue['#']] = issue
        return issues
    
    def _download_gantt(self, project):
        url = (self._proj_url % project) + "/gantt"
        url = uj(self._base_url, url)
        r1 = self._sess.get(url)
        r1.raise_for_status()
        return r1.content
        
        
class _Issue():
    
    def __init__(self, api, fields=None):
        self._fields = fields or OrderedDict()
        self._api = api
    
    def __getitem__(self, key):
        return self._fields[key]
    
    def __setitem__(self, key, value):
        self._fields[key] = value
    
    def items(self):
        return self._fields.items()
            
    def __repr__(self):
        return "Issue:\n" + "\n".join("%s: %s" % (k,v) for k, v in self.items())
    
    def download(self, type='pdf'):
        return self._api.download_issue(self['#'], type)d

In [11]:
i = IssuetrackerAPI("issue.pbsbiotech.com")
r2=i.login("nstarkweather@pbsbiotech.com", 'kookychemist')

In [85]:
q1 = pyquery.PyQuery(i._download_gantt('pbssoftware'))

In [86]:
q2 = q1("#content > table > tr > td > div > .gantt_subjects")
q3 = q2(".version-name")

In [87]:
for e in q3:
    el = pq(e)("span > a")
    if el.text() == '3.0':
        break

In [90]:
q2(".issue-subject")

[<div#issue-1921.issue-subject>, <div#issue-1959.issue-subject>, <div#issue-2712.issue-subject>, <div#issue-396.issue-subject>, <div#issue-420.issue-subject>, <div#issue-989.issue-subject>, <div#issue-2725.issue-subject>, <div#issue-1073.issue-subject>, <div#issue-2115.issue-subject>, <div#issue-2445.issue-subject>, <div#issue-2458.issue-subject>, <div#issue-2906.issue-subject>, <div#issue-2908.issue-subject>, <div#issue-2909.issue-subject>, <div#issue-1262.issue-subject>, <div#issue-1263.issue-subject>, <div#issue-1759.issue-subject>, <div#issue-1793.issue-subject>, <div#issue-1845.issue-subject>, <div#issue-1885.issue-subject>, <div#issue-1953.issue-subject>, <div#issue-2309.issue-subject>, <div#issue-3001.issue-subject>, <div#issue-2819.issue-subject>, <div#issue-2856.issue-subject>, <div#issue-2859.issue-subject>, <div#issue-2884.issue-subject>, <div#issue-2885.issue-subject>, <div#issue-2886.issue-subject>, <div#issue-2891.issue-subject>, <div#issue-2922.issue-subject>, <div#issue

In [57]:
pq = pyquery.PyQuery


In [22]:
csv = i.download_project_csv("pbssoftware")

In [23]:
print(csv[3264].download())

https://issue.pbsbiotech.com/issues/3264.pdf
b'%PDF-1.7\n3 0 obj\n<< /Type /Page /Parent 1 0 R /LastModified (D:20160824022231+02\'00\') /Resources 2 0 R /MediaBox [0.00 0.00 595.28 841.89] /CropBox [0.00 0.00 595.28 841.89] /BleedBox [0.00 0.00 595.28 841.89] /TrimBox [0.00 0.00 595.28 841.89] /ArtBox [0.00 0.00 595.28 841.89] /Contents 4 0 R /Rotate 0 /Group << /Type /Group /S /Transparency /CS /DeviceRGB >> /Annots [ 200001 0 R ] /PZ 1 >> endobj\n4 0 obj\n<</Filter /FlateDecode  /Length 3113>> stream\nx\x9c\xcd[mo$5\x12\xee\xcf\xf9\x15\x96N\xe8\x0e\tz\xedv\xdb\xed\xe6\x1b\xec\x0b\x02\x89\xb7\xdb\x9c\xee\x03\xa0SH\xb2\xd9\x85\xcd&$\xe1r\xfb\xef\xaf\xea\xa9\x97\xee\xb03\x9b\x89\x08\x11\xacf&\xcf\xb4\xc7.W\x95\xeb\xf1\xe3nb_\xa6p\x1db\xf8\x92^?\x87\xef\x7f\xa4\x8f#z}N\xaf\x93\xbd\xcf\xf6\xc3\xa3gCHC\x1fc\xd8\x7f\x11\x9e\xee\x87\xbdx\xcf\xbf\x18Z\x9fK\x98\xc6\xdc\xd71\x9c\x06\x85C\xedS\t\xaf\xc3s\xfe"N\xb8\x9e+]O\xc3\xd4\xb7b\x98\x1b\xdc\xd2?\xff`hK\x07\xc3<\xf5s^w\xc0\xdf\xd0\xd8\x8b\t

In [131]:
m1[0]
fields = m1[0].copy()
fields = [f.lower().replace(" ", "_").replace("%", "_").replace("/", "_") for f in fields]
s = "        def __init__(self, %s):"
s %= ", ".join(fields)
print(s)
s2 = " "*8+(" "*8).join("self._%s = %s\n" % (f, f) for f in fields)
print(s2)

        def __init__(self, #, project, tracker, parent_task, status, priority, subject, author, assignee, updated, category, sprint_milestone, start_date, due_date, estimated_time, spent_time, __done, created, closed, related_issues, type_of_request, product_type, serial_no._lot_no., followup_actions, return_material_authorization, root_cause, date_approved, private):
        self._# = #
        self._project = project
        self._tracker = tracker
        self._parent_task = parent_task
        self._status = status
        self._priority = priority
        self._subject = subject
        self._author = author
        self._assignee = assignee
        self._updated = updated
        self._category = category
        self._sprint_milestone = sprint_milestone
        self._start_date = start_date
        self._due_date = due_date
        self._estimated_time = estimated_time
        self._spent_time = spent_time
        self.___done = __done
        self._created = created
        sel